In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim

from stacksyms import parseELF, parseDWARF, getAllFunctions, collectFrameInfo, \
                      assign_frames, processRegisterRuleExpressions, propagateTypeInfo, \
                      collectOpcodes, collectDisassemblyObjdump, checkLabels

#filepath = 'data/binary/x86/gcc-32-O1-utillinux-unshare'
#filepath = 'data/cross-compile-dataset/bin/static/gcc/o1/pee'
#filepath = 'data/cross-compile-dataset/bin/static/gcc/og/parallel'
#filedir = 'data/cross-compile-dataset/bin/static/gcc/og'
#filepath = 'data/cross-compile-dataset/bin/dynamic/clang/o3/xrdb'
#filedir = '.'
#filepath = 'data/cross-compile-dataset/bin/static/gcc/og/sum'
#filepath = 'data/spec2006/bzip2'

In [3]:
functions = parseELF(filepath)
print(len(functions))

05/16/2021 19:34:37 INFO:parseELF:Trying to parse data/cross-compile-dataset/bin/static/gcc/o1/pee as ELF
05/16/2021 19:34:37 INFO:parseELF:ELF file is for architecture x64.
05/16/2021 19:34:37 INFO:parseDWARF:ELF file says it has some frame info..
05/16/2021 19:34:37 INFO:getFunctionsFromSymtab:Trying to obtain symbol information.
05/16/2021 19:34:37 INFO:getFunctionsFromSymtab:Found 869 functions according to symbol table
05/16/2021 19:34:37 INFO:getFunctionsFromDWARFInfo:Searching for subroutines with explicit DWARF info..
05/16/2021 19:34:37 INFO:collectFrameInfo:has .eh_frames
05/16/2021 19:34:38 WARNING:collectFrameInfo:FDE/CIE for address 0x4230d0 without matching symbol.
05/16/2021 19:34:38 WARNING:collectFrameInfo:FDE/CIE for address 0x423310 without matching symbol.
05/16/2021 19:34:38 WARNING:collectFrameInfo:FDE/CIE for address 0x425530 without matching symbol.
05/16/2021 19:34:38 WARNING:collectFrameInfo:FDE/CIE for address 0x425640 without matching symbol.
05/16/2021 19:3

869


In [4]:
#import logging; logging.getLogger().setLevel(logging.DEBUG)
from stacksyms import getMaxFrameSize, getMaxFrameSizeCFA, generateDebugLabel, getStackElements, getStackLocations, getMaxStackOff
doesntwork = []
for func in functions:
    cfa   = getMaxFrameSizeCFA(func)
    frame = getMaxFrameSize(func)
    label = generateDebugLabel(func)
    if None in label:
        print("BUG!!!!!!!!", func.name)
        print([(stkElm, stkElm.type.qualified_name, stkElm.type.byte_size) for stkElm in getStackElements(func)])
        break
    if sum(label)+8 != frame:
        doesntwork += [func]
    print(f"{func.name} (frame starts at {func.frame_base}, size is {cfa}/{frame}/{8+sum(label)} bytes by cfa/offset/sum) => {label}")
    if 0<len(func.inlined_functions):
        print("    inlines ", func.inlined_functions)
        for inlined in func.inlined_functions:
            print("        ", inlined.name, getMaxFrameSize(inlined), generateDebugLabel(inlined))
        break

05/16/2021 19:34:44 INFO:generateDebugLabel:Function backtrace_and_maps has 3 stack elements out of 4 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function __libc_message.constprop.0 has 6 stack elements out of 7 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function mem2chunk_check has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function __malloc_assert has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function arena_thread_freeres has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function new_heap has 3 stack elements out of 4 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function mremap_chunk has 4 stack elements out of 5 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function detach_arena.part.0 has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function get_free_list has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDe

05/16/2021 19:34:44 INFO:generateDebugLabel:Function free_mem has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function insert_module has 5 stack elements out of 6 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function add_module.isra.0 has 6 stack elements out of 7 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function find_module has 6 stack elements out of 7 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function free_mem has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function known_compare has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function do_release_all has 1 stack elements out of 2 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function free_mem has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function do_release_shlib has 1 stack elements out of 2 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function new_composite_name h

05/16/2021 19:34:44 INFO:generateDebugLabel:Function str_to_mpn.isra.0 has 6 stack elements out of 7 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function round_and_return has 6 stack elements out of 7 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function str_to_mpn.isra.0 has 6 stack elements out of 7 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function round_and_return has 6 stack elements out of 7 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function read_encoded_value_with_base has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function base_of_encoded_value has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function execute_cfa_program has 6 stack elements out of 7 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function init_dwarf_reg_size_table has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function uw_frame_state_for has 6 stack elements out of 7 total.
05/1

backtrace_and_maps (frame starts at None, size is 1568/1568/32 bytes by cfa/offset/sum) => [8, 8, 8]
__libc_message.constprop.0 (frame starts at None, size is 16/56/56 bytes by cfa/offset/sum) => [8, 8, 8, 8, 8, 8]
mem2chunk_check (frame starts at None, size is 8/8/8 bytes by cfa/offset/sum) => []
__malloc_assert (frame starts at None, size is 48/48/8 bytes by cfa/offset/sum) => []
arena_thread_freeres (frame starts at None, size is 136/136/8 bytes by cfa/offset/sum) => []
new_heap (frame starts at None, size is 32/32/32 bytes by cfa/offset/sum) => [8, 8, 8]
mremap_chunk (frame starts at None, size is 48/48/40 bytes by cfa/offset/sum) => [8, 8, 8, 8]
detach_arena.part.0 (frame starts at None, size is 16/16/8 bytes by cfa/offset/sum) => []
get_free_list (frame starts at None, size is 144/144/8 bytes by cfa/offset/sum) => []
malloc_printerr (frame starts at None, size is 80/80/48 bytes by cfa/offset/sum) => [8, 8, 8, 8, 8]
systrim.isra.2 (frame starts at None, size is 48/48/48 bytes by c

05/16/2021 19:34:44 INFO:generateDebugLabel:Function sysconf has 2 stack elements out of 3 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function __strtod_internal has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function _IO_seekoff_unlocked has 2 stack elements out of 3 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function _nl_load_domain has 6 stack elements out of 7 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function _dl_runtime_resolve_avx has 1 stack elements out of 2 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function _IO_default_doallocate has 3 stack elements out of 4 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function __strtoull_l has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function fdopendir has 1 stack elements out of 2 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function _dl_important_hwcaps has 6 stack elements out of 7 total.
05/16/2021 19:34:44 INFO:generateDeb

...

05/16/2021 19:34:44 INFO:generateDebugLabel:Function _Unwind_SetIP has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function __libc_csu_init has 5 stack elements out of 6 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function _dl_unmap has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function __memset_avx512_unaligned has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function __get_nprocs_conf has 3 stack elements out of 4 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function __gconv_release_step has 2 stack elements out of 3 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function _IO_wdefault_xsputn has 6 stack elements out of 7 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function __gconv_transform_internal_utf8 has 6 stack elements out of 7 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function localtime has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INF

_IO_file_stat (frame starts at None, size is 8/8/8 bytes by cfa/offset/sum) => []
_dl_start (frame starts at None, size is 16/16/8 bytes by cfa/offset/sum) => []
malloc_usable_size (frame starts at None, size is 80/80/40 bytes by cfa/offset/sum) => [8, 8, 8, 8]
__sscanf (frame starts at None, size is 224/224/8 bytes by cfa/offset/sum) => []
__strtold_internal (frame starts at None, size is 8/8/8 bytes by cfa/offset/sum) => []
__tzfile_default (frame starts at None, size is 96/96/56 bytes by cfa/offset/sum) => [8, 8, 8, 8, 8, 8]
__register_frame_info_bases (frame starts at None, size is 16/16/16 bytes by cfa/offset/sum) => [8]
_IO_wfile_sync (frame starts at None, size is 48/48/32 bytes by cfa/offset/sum) => [8, 8, 8]
strtod (frame starts at None, size is 8/8/8 bytes by cfa/offset/sum) => []
_IO_vfscanf_internal (frame starts at None, size is 144/144/56 bytes by cfa/offset/sum) => [8, 8, 8, 8, 8, 8]
__readonly_area (frame starts at None, size is 80/80/48 bytes by cfa/offset/sum) => [8, 

05/16/2021 19:34:44 INFO:generateDebugLabel:Function _IO_init_wmarker has 2 stack elements out of 3 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function _dl_runtime_resolve_sse has 1 stack elements out of 2 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function setlocale has 6 stack elements out of 7 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function __getclktck has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function _Unwind_GetTextRelBase has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function _IO_file_read has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function __libc_setup_tls has 6 stack elements out of 7 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function __strtold_nan has 2 stack elements out of 3 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function _dl_runtime_profile_avx has 1 stack elements out of 2 total.
05/16/2021 19:34:44 INFO:generateDe

05/16/2021 19:34:44 INFO:generateDebugLabel:Function _IO_wsetb has 1 stack elements out of 2 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function __tzstring has 5 stack elements out of 6 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function __wcsmbs_named_conv has 2 stack elements out of 3 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function _IO_seekoff has 5 stack elements out of 6 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function _dl_aux_init has 6 stack elements out of 7 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function __memset_sse2_unaligned_erms has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function _IO_wfile_doallocate has 2 stack elements out of 3 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function __assert_fail_base has 5 stack elements out of 6 total.
05/16/2021 19:34:44 INFO:generateDebugLabel:Function __strcasecmp_ssse3 has 0 stack elements out of 1 total.
05/16/2021 19:34:44 INFO:generateDeb

__gconv_compare_alias_cache (frame starts at None, size is 112/112/56 bytes by cfa/offset/sum) => [8, 8, 8, 8, 8, 8]
_IO_init_wmarker (frame starts at None, size is 32/32/24 bytes by cfa/offset/sum) => [8, 8]
_dl_runtime_resolve_sse (frame starts at None, size is 288/288/16 bytes by cfa/offset/sum) => [8]
setlocale (frame starts at None, size is 320/320/56 bytes by cfa/offset/sum) => [8, 8, 8, 8, 8, 8]
__getclktck (frame starts at None, size is 8/8/8 bytes by cfa/offset/sum) => []
_Unwind_GetTextRelBase (frame starts at None, size is 8/8/8 bytes by cfa/offset/sum) => []
_IO_file_read (frame starts at None, size is 8/8/8 bytes by cfa/offset/sum) => []
__libc_setup_tls (frame starts at None, size is 96/96/56 bytes by cfa/offset/sum) => [8, 8, 8, 8, 8, 8]
__strtold_nan (frame starts at None, size is 64/64/24 bytes by cfa/offset/sum) => [8, 8]
_dl_runtime_profile_avx (frame starts at None, size is 56/56/16 bytes by cfa/offset/sum) => [8]
__strncasecmp_sse42 (frame starts at None, size is 8

In [ ]:
# In principle, on x86 we can read out the stack size of a function from the
# .eh_frame section as follows:
# 1) "nm elf_binary | grep function_name" -> note address
# 2) "readelf --debug-dump=frames-interp | grep -A10 'address..'"
#    -> read off maximum stack offset from last entry under CFA column
# The problem is that this doesn't work on architectures that do not store the
# return address on the stack and even on x86 some compiler passes may result in
# code that will try to hold the return address in a register, even though it is
# stored on the stack (e.g., "gcc-32-O1-utillinux-unshare")
from elftools.elf.elffile import ELFFile

elf = ELFFile(open(filepath, 'rb'))
module, importer = parseDWARF(elf)
func_dict = getAllFunctions(module, importer, elf)

In [ ]:
frame_tables = collectFrameInfo(func_dict, elf)
func_dict = assign_frames(frame_tables, func_dict)
func_dict = processRegisterRuleExpressions(func_dict, importer)
func_dict = propagateTypeInfo(func_dict, importer)
func_dict = collectOpcodes(func_dict, elf)
#labels = checkLabels(functions)

In [ ]:
works = [func for func in functions if func not in doesntwork]
works

In [ ]:
func = next(filter(lambda func : func.name=='_dl_sort_fini', functions))
func_stack = sorted(getStackElements(func), key=getMaxStackOff)
print(func_stack, generateDebugLabel(func), getMaxFrameSize(func), sum(generateDebugLabel(func)))
print(func.registers)
#quotearg_n_style_stack[4]


In [ ]:
quotearg_n_style = next(filter(lambda func : func.name=='quotearg_n_style', functions))
quotearg_n_style_stack = sorted(getStackElements(quotearg_n_style), key=getMaxStackOff)
print(quotearg_n_style_stack)
#getStackLocations(quotearg_n_style_stack[0])
quotearg_n_style_stack[4]


In [1]:
import os
import json
from stacksyms import parseELF, checkLabels
dataroot  = "data/cross-compile-dataset/bin/static/gcc/o2"
outputdir = "data/cross-compile-dataset/labels/static/gcc/o2"
f = 'yes'
filepath  = os.path.join(dataroot, f)
functions = parseELF(filepath)
allLabels = checkLabels(functions)
#if not os.path.exists(outputdir):
#    os.makedirs(outputdir, exist_ok=True)
#with open(os.path.join(outputdir, f), 'w') as cf:
#    json.dump(allLabels, cf)

05/16/2021 19:34:26 INFO:parseELF:Trying to parse data/cross-compile-dataset/bin/static/gcc/o2/yes as ELF
05/16/2021 19:34:26 INFO:parseELF:ELF file is for architecture x64.
05/16/2021 19:34:26 INFO:parseDWARF:ELF file says it has some frame info..
05/16/2021 19:34:26 INFO:getFunctionsFromSymtab:Trying to obtain symbol information.
05/16/2021 19:34:26 INFO:getFunctionsFromSymtab:Found 963 functions according to symbol table
05/16/2021 19:34:26 INFO:getFunctionsFromDWARFInfo:Searching for subroutines with explicit DWARF info..
05/16/2021 19:34:26 INFO:getFunctionsFromDWARFInfo:Searching for subroutines with explicit DWARF info..
05/16/2021 19:34:26 INFO:getFunctionsFromDWARFInfo:Searching for subroutines with explicit DWARF info..
05/16/2021 19:34:26 INFO:collectFrameInfo:has .eh_frames
05/16/2021 19:34:27 WARNING:collectFrameInfo:FDE/CIE for address 0x428340 without matching symbol.
05/16/2021 19:34:27 WARNING:collectFrameInfo:FDE/CIE for address 0x428580 without matching symbol.
05/16

05/16/2021 19:34:28 INFO:checkLabels:detach_arena.part.0 (16 by offset / 0 by size) => []
05/16/2021 19:34:28 INFO:generateDebugLabel:Function get_free_list has 0 stack elements out of 1 total.
05/16/2021 19:34:28 WARNING:checkLabels:Label generation for get_free_list is not sound (got 144 by offset, but 0 by size: [])!
05/16/2021 19:34:28 INFO:checkLabels:get_free_list (144 by offset / 0 by size) => []
05/16/2021 19:34:28 INFO:generateDebugLabel:Function malloc_printerr has 5 stack elements out of 6 total.
05/16/2021 19:34:28 WARNING:checkLabels:Label generation for malloc_printerr is not sound (got 80 by offset, but 40 by size: [8, 8, 8, 8, 8])!
05/16/2021 19:34:28 INFO:checkLabels:malloc_printerr (80 by offset / 40 by size) => [8, 8, 8, 8, 8]
05/16/2021 19:34:28 INFO:generateDebugLabel:Function systrim.isra.2 has 5 stack elements out of 6 total.
05/16/2021 19:34:28 INFO:checkLabels:systrim.isra.2 (48 by offset / 40 by size) => [8, 8, 8, 8, 8]
05/16/2021 19:34:28 INFO:generateDebugLa

05/16/2021 19:34:28 INFO:generateDebugLabel:Function __gconv_release_step.part.1 has 0 stack elements out of 1 total.
05/16/2021 19:34:28 WARNING:checkLabels:Label generation for __gconv_release_step.part.1 is not sound (got 16 by offset, but 0 by size: [])!
05/16/2021 19:34:28 INFO:checkLabels:__gconv_release_step.part.1 (16 by offset / 0 by size) => []
05/16/2021 19:34:28 INFO:generateDebugLabel:Function free_modules_db has 6 stack elements out of 7 total.
05/16/2021 19:34:28 WARNING:checkLabels:Label generation for free_modules_db is not sound (got 64 by offset, but 48 by size: [8, 8, 8, 8, 8, 8])!
05/16/2021 19:34:28 INFO:checkLabels:free_modules_db (64 by offset / 48 by size) => [8, 8, 8, 8, 8, 8]
05/16/2021 19:34:28 INFO:generateDebugLabel:Function free_mem has 6 stack elements out of 7 total.
05/16/2021 19:34:28 WARNING:checkLabels:Label generation for free_mem is not sound (got 64 by offset, but 48 by size: [8, 8, 8, 8, 8, 8])!
05/16/2021 19:34:28 INFO:checkLabels:free_mem (64 

05/16/2021 19:34:28 INFO:generateDebugLabel:Function adjust_wide_data has 3 stack elements out of 4 total.
05/16/2021 19:34:28 WARNING:checkLabels:Label generation for adjust_wide_data is not sound (got 64 by offset, but 24 by size: [8, 8, 8])!
05/16/2021 19:34:28 INFO:checkLabels:adjust_wide_data (64 by offset / 24 by size) => [8, 8, 8]
05/16/2021 19:34:28 INFO:generateDebugLabel:Function _IO_wfile_underflow_mmap has 2 stack elements out of 3 total.
05/16/2021 19:34:28 WARNING:checkLabels:Label generation for _IO_wfile_underflow_mmap is not sound (got 64 by offset, but 16 by size: [8, 8])!
05/16/2021 19:34:28 INFO:checkLabels:_IO_wfile_underflow_mmap (64 by offset / 16 by size) => [8, 8]
05/16/2021 19:34:28 INFO:generateDebugLabel:Function _IO_wfile_underflow_maybe_mmap has 1 stack elements out of 2 total.
05/16/2021 19:34:28 INFO:checkLabels:_IO_wfile_underflow_maybe_mmap (16 by offset / 8 by size) => [8]
05/16/2021 19:34:28 INFO:generateDebugLabel:Function _IO_file_seekoff_maybe_mma

05/16/2021 19:34:28 WARNING:checkLabels:Label generation for free_mem is not sound (got 144 by offset, but 0 by size: [])!
05/16/2021 19:34:28 INFO:checkLabels:free_mem (144 by offset / 0 by size) => []
05/16/2021 19:34:28 INFO:generateDebugLabel:Function read_int has 0 stack elements out of 1 total.
05/16/2021 19:34:28 INFO:checkLabels:read_int (8 by offset / 0 by size) => []
05/16/2021 19:34:28 INFO:generateDebugLabel:Function group_number has 6 stack elements out of 7 total.
05/16/2021 19:34:28 INFO:checkLabels:group_number (56 by offset / 48 by size) => [8, 8, 8, 8, 8, 8]
05/16/2021 19:34:28 INFO:generateDebugLabel:Function _i18n_number_rewrite has 6 stack elements out of 7 total.
05/16/2021 19:34:28 INFO:checkLabels:_i18n_number_rewrite (56 by offset / 48 by size) => [8, 8, 8, 8, 8, 8]
05/16/2021 19:34:28 INFO:generateDebugLabel:Function _IO_helper_overflow has 4 stack elements out of 5 total.
05/16/2021 19:34:28 WARNING:checkLabels:Label generation for _IO_helper_overflow is not 

...

05/16/2021 19:34:29 WARNING:checkLabels:Label generation for __strtold_nan is not sound (got 64 by offset, but 16 by size: [8, 8])!
05/16/2021 19:34:29 INFO:checkLabels:__strtold_nan (64 by offset / 16 by size) => [8, 8]
05/16/2021 19:34:29 INFO:generateDebugLabel:Function _dl_runtime_profile_avx has 1 stack elements out of 2 total.
05/16/2021 19:34:29 WARNING:checkLabels:Label generation for _dl_runtime_profile_avx is not sound (got 56 by offset, but 8 by size: [8])!
05/16/2021 19:34:29 INFO:checkLabels:_dl_runtime_profile_avx (56 by offset / 8 by size) => [8]
05/16/2021 19:34:29 INFO:generateDebugLabel:Function __strncasecmp_sse42 has 0 stack elements out of 1 total.
05/16/2021 19:34:29 INFO:checkLabels:__strncasecmp_sse42 (8 by offset / 0 by size) => []
05/16/2021 19:34:29 INFO:checkLabels:__close_nocancel (0 by offset / 0 by size) => []
05/16/2021 19:34:29 INFO:generateDebugLabel:Function __stpcpy_ssse3 has 0 stack elements out of 1 total.
05/16/2021 19:34:29 INFO:checkLabels:__stp

05/16/2021 19:34:29 INFO:generateDebugLabel:Function fwrite_unlocked has 4 stack elements out of 5 total.
05/16/2021 19:34:29 WARNING:checkLabels:Label generation for fwrite_unlocked is not sound (got 64 by offset, but 32 by size: [8, 8, 8, 8])!
05/16/2021 19:34:29 INFO:checkLabels:fwrite_unlocked (64 by offset / 32 by size) => [8, 8, 8, 8]
05/16/2021 19:34:29 INFO:generateDebugLabel:Function _IO_unsave_wmarkers has 1 stack elements out of 2 total.
05/16/2021 19:34:29 INFO:checkLabels:_IO_unsave_wmarkers (16 by offset / 8 by size) => [8]
05/16/2021 19:34:29 INFO:generateDebugLabel:Function _IO_file_open has 3 stack elements out of 4 total.
05/16/2021 19:34:29 INFO:checkLabels:_IO_file_open (32 by offset / 24 by size) => [8, 8, 8]
05/16/2021 19:34:29 INFO:generateDebugLabel:Function _dl_map_object has 6 stack elements out of 7 total.
05/16/2021 19:34:29 WARNING:checkLabels:Label generation for _dl_map_object is not sound (got 1024 by offset, but 48 by size: [8, 8, 8, 8, 8, 8])!
05/16/20

05/16/2021 19:34:29 INFO:generateDebugLabel:Function __dlerror has 2 stack elements out of 3 total.
05/16/2021 19:34:29 WARNING:checkLabels:Label generation for __dlerror is not sound (got 48 by offset, but 16 by size: [8, 8])!
05/16/2021 19:34:29 INFO:checkLabels:__dlerror (48 by offset / 16 by size) => [8, 8]
05/16/2021 19:34:29 INFO:generateDebugLabel:Function exit has 0 stack elements out of 1 total.
05/16/2021 19:34:29 WARNING:checkLabels:Label generation for exit is not sound (got 16 by offset, but 0 by size: [])!
05/16/2021 19:34:29 INFO:checkLabels:exit (16 by offset / 0 by size) => []
05/16/2021 19:34:29 INFO:generateDebugLabel:Function _Unwind_SetGR has 0 stack elements out of 1 total.
05/16/2021 19:34:29 WARNING:checkLabels:Label generation for _Unwind_SetGR is not sound (got 16 by offset, but 0 by size: [])!
05/16/2021 19:34:29 INFO:checkLabels:_Unwind_SetGR (16 by offset / 0 by size) => []
05/16/2021 19:34:29 INFO:generateDebugLabel:Function __gconv_transform_internal_ucs4

05/16/2021 19:34:29 INFO:checkLabels:__strcmp_ssse3 (8 by offset / 0 by size) => []
05/16/2021 19:34:29 INFO:generateDebugLabel:Function _dl_tlsdesc_return has 0 stack elements out of 1 total.
05/16/2021 19:34:29 INFO:checkLabels:_dl_tlsdesc_return (8 by offset / 0 by size) => []
05/16/2021 19:34:29 INFO:generateDebugLabel:Function __alloc_dir has 3 stack elements out of 4 total.
05/16/2021 19:34:29 WARNING:checkLabels:Label generation for __alloc_dir is not sound (got 48 by offset, but 24 by size: [8, 8, 8])!
05/16/2021 19:34:29 INFO:checkLabels:__alloc_dir (48 by offset / 24 by size) => [8, 8, 8]
05/16/2021 19:34:29 INFO:generateDebugLabel:Function __strcasecmp_sse42 has 0 stack elements out of 1 total.
05/16/2021 19:34:29 INFO:checkLabels:__strcasecmp_sse42 (8 by offset / 0 by size) => []
05/16/2021 19:34:29 INFO:generateDebugLabel:Function __strcasecmp_avx has 0 stack elements out of 1 total.
05/16/2021 19:34:29 INFO:checkLabels:__strcasecmp_avx (8 by offset / 0 by size) => []
05/1

05/16/2021 19:34:29 INFO:checkLabels:__strcmp_sse2_unaligned (8 by offset / 0 by size) => []
05/16/2021 19:34:29 INFO:generateDebugLabel:Function _dl_catch_error has 1 stack elements out of 2 total.
05/16/2021 19:34:29 WARNING:checkLabels:Label generation for _dl_catch_error is not sound (got 336 by offset, but 8 by size: [8])!
05/16/2021 19:34:29 INFO:checkLabels:_dl_catch_error (336 by offset / 8 by size) => [8]
05/16/2021 19:34:29 INFO:generateDebugLabel:Function _IO_un_link has 2 stack elements out of 3 total.
05/16/2021 19:34:29 WARNING:checkLabels:Label generation for _IO_un_link is not sound (got 192 by offset, but 16 by size: [8, 8])!
05/16/2021 19:34:29 INFO:checkLabels:_IO_un_link (192 by offset / 16 by size) => [8, 8]
05/16/2021 19:34:29 INFO:generateDebugLabel:Function __register_frame_info_table has 0 stack elements out of 1 total.
05/16/2021 19:34:29 INFO:checkLabels:__register_frame_info_table (8 by offset / 0 by size) => []
05/16/2021 19:34:29 INFO:generateDebugLabel:Fu

In [ ]:
hex(4208045)

In [ ]:
#PAD_TOK8  = 0xa1 # 1 byte isn't super reliable.. might actually appear in instructions
PAD_TOK16 =  0x0f78 # vmread (0x0f79 is vmwrite, could also use 'UD2' 0x0f0b)
TOK_LEN = 2 # a token is two bytes
PAD_TOK = PAD_TOK16
MAX_INST_LEN = 16 # pretty sure 128 bits is the limit on x86

def tokenize(features):
    return [[int(x[i:i+2*TOK_LEN], 16) for i in range(0,len(x),2*TOK_LEN)] for x in features]

def funcFeatures(function): # features are instructions (16 bytes max)
    return tokenize(map(lambda t : t[1].replace(' ', ''), function.disas))

def generatePositionalEncodings(function): # code addresses
    return list(map(lambda t : t[0][0:18], function.disas))

for func in functions:
    if 0<len(func.disas):
        print()

In [ ]:
X, Y, Z = [], [], []
for func in functions:
    print('////////////////////////')
    print(func.name, hex(func.start), func.frame_base)
    for lvar in func.variables:
        print(lvar.name, lvar.type, "(bytesize = %d)"%lvar.type.byte_size)
        for loc in lvar.locations:
            print(hex(loc.begin) + " to " + hex(loc.end) + ": " \
            + str(loc.type)[13:]
            + str(loc.expr))
        print('')
#    X += [func] #generateFeature(func, functions)]
#    Y += [generateLabel(func, functions)]
#    Z += [generateDebugLabel(func, functions)]
#print([x+" => "+ str(y) for x,y in zip(X,Z)])

#func = 'quotearg_n_style_colon'

In [ ]:
'''Doesnt appear to be super useful for stack symbolization as compilers dont seem to emit relevant information'''
def handleLineprogram(dwarfInfo):
    print('looping through compilation units..')
    for cu in dwarfInfo.iter_CUs():
        lp = dwarfInfo.line_program_for_CU(cu)
        if lp == None:
            print('DWARF info is missing a line program for this CU')
            print(cu.keys())
            continue
        cu_filename = lp['file_entry'][0].name.decode('latin-1')
        if len(lp['include_directory']) > 0:
            dir_index = lp['file_entry'][0].dir_index
            if dir_index > 0:
                idir = lp['include_directory'][dir_index - 1]
            else:
                idir = b'.'
            cu_filename = '%s/%s' % (idir.decode('latin-1'), cu_filename)
        print(f'CU: {cu_filename}')
        #print(f'File name                            Line number    Starting address')
        for entry in lp.get_entries():
            '''
            if entry.state is None:
                # Special handling for commands that don't set a new state
                if entry.command == DW_LNS_set_file:
                    file_entry = lp['file_entry'][entry.args[0] - 1]
                    if file_entry.dir_index == 0:
                        # current directory
                        print('\n./%s:[++]' % (
                            file_entry.name.decode('latin-1')))
                    else:
                        print('\n%s/%s:' % (
                            lp['include_directory'][file_entry.dir_index - 1].decode('latin-1'),
                            file_entry.name.decode('latin-1')))
                elif entry.command == DW_LNE_define_file:
                    print('%s:' % (
                        lp['include_directory'][entry.args[0].dir_index].decode('latin-1')))
                elif entry.command in [DW_LNS_set_basic_block, DW_LNS_set_prologue_end, DW_LNS_set_isa]:
                    print('=============================')
                    print(repr(entry))
                    print('=============================')
                else:
                    print('=============================')
                    print('OTHER COMMAND: ' + str(entry))
                    print('=============================')
            elif not entry.state.end_sequence:
                # readelf doesn't print the state after end_sequence
                # instructions. I think it's a bug but to be compatible
                # I don't print them too.
                if lp['version'] < 4:
                    print('%-35s  %11d  %18s' % (
                        lp['file_entry'][state.file - 1].name.decode('latin-1'),
                        state.line,
                        '0' if state.address == 0 else
                            hex(state.address)))
                else:
                    print('%-35s  %11d  %18s[%d]' % (
                        lp['file_entry'][state.file - 1].name.decode('latin-1'),
                        state.line,
                        '0' if state.address == 0 else
                            hex(state.address),
                        state.op_index))
            if entry.command == DW_LNS_copy:
                # Another readelf oddity...
                print()
            '''
            if entry.command in [DW_LNS_set_basic_block, DW_LNS_set_prologue_end, DW_LNS_set_isa]:
                print('=============================')
                print(repr(entry))
                print('=============================')
        #print(dir(lp))
        #line_entry_mapping(lp)

def line_entry_mapping(line_program):
    import collections
    filename_map = collections.defaultdict(int)
    lp_entries = line_program.get_entries()
    for lpe in lp_entries:
        if not lpe.state or lpe.state.file == 0:
            continue # TODO: instruction doesn't correspond to src
        filename = lpe_filename(line_program, lpe.state.file)
        filename_map[filename] += 1
    for filename, lpe_count in filename_map.items():
        print('%s -> %d entries' % (filename, lpe_count))

def lpe_filename(line_program, file_index):
    lp_header = line_program.header
    print(lp_header)
    file_entries = lp_header["file_entry"]
    file_entry = file_entries[file_index - 1]
    dir_index = file_entry["dir_index"]
    if dir_index == 0:
        return file_entry.name.decode()
    directory = lp_header["include_directory"][dir_index - 1]
    return os.path.join(directory, file_entry.name).decode()

    

#print(elf.get_section_by_name('.text').data())